# QA, QC, Metadata

The purpose of this notebook is for ensuring the `modis_wrf_aligned` dataset meets SNAPs standards for metadata, quality assurance, and quality control. 

## Metadata

### Bounding coordinates

First, are WRF and MODIS the same?

In [1]:
import os, glob, datetime
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
from rasterio.crs import CRS
from rasterio.warp import transform_bounds

out_dir = os.getenv("OUTPUT_DIR")
wrf_src = rio.open(os.path.join(out_dir, "WRF", "tsk_1km_3338_multiband", "tsk_ccsm_mean_MOD11A2_multiband.tif"))
mod_src = rio.open(os.path.join(out_dir, "MODIS", "lst_1km_3338_multiband", "lst_MOD11A2_InteriorAK_3338_multiband.tif"))

print("WRF bounds (epsg:3338):", list(wrf_src.bounds))
print("MODIS bounds (epsg:3338):", list(mod_src.bounds))

WRF bounds (epsg:3338): [189534.91234932584, 1539916.5811932846, 552947.9944114918, 1800699.1169323388]
MODIS bounds (epsg:3338): [189534.91234932584, 1539916.5811932846, 552947.9944114918, 1800699.1169323388]


Yes. Convert bounds to EPSG:4326:

In [2]:
bb = transform_bounds(wrf_src.crs, CRS.from_epsg(4326), *wrf_src.bounds)
print("Western bound:", bb[0])
print("Southern bound:", bb[1])
print("Eastern bound:", bb[2])
print("Northern bound:", bb[3])

Western bound: -150.14808420148256
Southern bound: 63.40847592793152
Eastern bound: -141.90837126457137
Northern bound: 66.11036303740029


In [3]:
mod_src.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -9999.0,
 'width': 216,
 'height': 155,
 'count': 931,
 'crs': CRS.from_epsg(3338),
 'transform': Affine(1682.4679725100275, 0.0, 189534.91234932584,
        0.0, -1682.467972510027, 1800699.1169323388)}

### "Attribute table"

Here, attribute is taken to mean a data field. Attributes are different depending on the format, since there are three distinct formats: single band GeoTIFFs, multiband GeoTIFFs, and NetCDFs. 

##### GeoTIFFs

There are no atributes for the GeoTIFFs, just the data and metadata, which are implied from the format (i.e., not labeled). 

##### NetCDFs


In [4]:
ds = xr.open_dataset(os.path.join(out_dir, "WRF", "tsk_1km_3338_multiband", "tsk_era_mean_MOD11A2_multiband.nc"))

In [5]:
ds.var

<bound method ImplementsDatasetReduce._reduce_method.<locals>.wrapped_func of <xarray.Dataset>
Dimensions:  (date: 866, xc: 216, yc: 155)
Coordinates:
  * xc       (xc) float64 1.904e+05 1.921e+05 1.937e+05 ... 5.504e+05 5.521e+05
  * yc       (yc) float64 1.8e+06 1.798e+06 1.796e+06 ... 1.542e+06 1.541e+06
  * date     (date) datetime64[ns] 2000-02-18 2000-02-26 ... 2018-12-19
Data variables:
    tsk      (date, yc, xc) float32 ...
Attributes:
    title:          1km MODIS-ailgned WRF TSK
    creation_time:  2020-08-05 14:15:35
    epsg:           3338
    nc_source:      WRFDS_2000-02-21_serdp.nc
    SNAP_version:   0.1.0>

The NetCDF files have three dimensions and a data variable. The data variable is either "tsk" (WRF) or "lst" (MODIS), and the dimensions are "xc", "yc", and "date". These correspond to the x and y coordinate in EPSG:3338, and the date in YYYY-mm-dd format.

### Spatial reference

All data are in EPSG:3338, included as metadata in all files:

In [6]:
print("Single band GeoTIFF CRS:", mod_src.crs)

Single band GeoTIFF CRS: EPSG:3338


In [7]:
print("NetCDF CRS: EPSG:", ds.attrs["epsg"])

NetCDF CRS: EPSG: 3338


### Source data

##### time period

Timespan of source MODIS data is contained within timespan of the downscaled data. Thus, timespan of the output data gives the timespan of the MODIS data:

In [8]:
print("MODIS begin date:", min(ds.date.values))
print("MODIS end date:", max(ds.date.values))

MODIS begin date: 2000-02-18T00:00:00.000000000
MODIS end date: 2018-12-19T00:00:00.000000000


## QA/QC

### Missing data

Check that there are no missing files or missing bands from the multiband files.

First, verify that differences between dates in single band filenames are no greater than 8 days.

In [63]:
# find gaps in time series created from single band filenames
# returns dates where a gap was longer than 8 days
def find_gaps(sb_dir):
    fps = sorted(glob.glob(os.path.join(sb_dir, "*")))
    jds = [fp.split("_")[-2] for fp in fps]
    dates = [datetime.datetime.strptime(jd, "%Y%j").date() for jd in jds]
    diffs = pd.Series(dates).diff().iloc[1:]
    diffs = np.array([diff.days for diff in diffs])
    idx = np.where(diffs > 8)
    if len(idx[0]) > 0:
        return [int(i) for i in idx]
    else:
        return 0


In [76]:
find_gaps(wrf_dirs[0])

[60]

In [74]:
find_gaps(wrf_dir)

0

In [15]:
wrf_sb_dir = os.path.join(out_dir, "WRF", "tsk_1km_3338", "*")
wrf_dirs = glob.glob(wrf_sb_dir)

In [73]:
gaps = []
for wrf_dir in wrf_dirs:
    print(wrf_dir)
    gaps += find_gaps(wrf_dir)

/home/kmredilla/projects/SERDP-fish-fire/data/WRF/tsk_1km_3338/era_mean_MOD11A2
/home/kmredilla/projects/SERDP-fish-fire/data/WRF/tsk_1km_3338/era_min_MOD11A2
/home/kmredilla/projects/SERDP-fish-fire/data/WRF/tsk_1km_3338/era_max_MOD11A2
/home/kmredilla/projects/SERDP-fish-fire/data/WRF/tsk_1km_3338/era_mean_MYD11A2


TypeError: 'int' object is not iterable

In [67]:
wrf_dir

'/home/kmredilla/projects/SERDP-fish-fire/data/WRF/tsk_1km_3338/era_mean_MYD11A2'

In [19]:
sb_dir = wrf_dirs[0]

In [96]:
print("Are there any gaps of greater than 8 days? ", )

[datetime.date(2001, 5, 25),
 datetime.date(2001, 6, 2),
 datetime.date(2001, 6, 10),
 datetime.date(2001, 6, 26),
 datetime.date(2001, 7, 4)]

In [92]:
np.where(diffs > 8)

(array([60]),)

In [85]:
!ls 

8

In [51]:
pd.Series(jds).diff().iloc[1:]

1      8.0
2      8.0
3      8.0
4      8.0
5      8.0
      ... 
861    8.0
862    8.0
863    8.0
864    8.0
865    8.0
Length: 865, dtype: float64

In [45]:
any(pd.Series(jds).diff().iloc[1:] > 8)

True

In [26]:
meta_df.RANGEBEGINNINGDATE

0       2000-02-18
1       2000-02-26
2       2000-03-05
3       2000-03-13
4       2000-03-21
           ...    
1749    2020-04-14
1750    2020-04-22
1751    2020-04-30
1752    2020-05-08
1753    2020-05-16
Name: RANGEBEGINNINGDATE, Length: 1754, dtype: object

In [21]:
!ls /center1/DYNDOWN/kmredilla/data/modis_lst-scratch/ancillary/MODIS_LST_8dayComposite_begin_end_range_metadata_final.csv

MODIS_LST_8dayComposite_begin_end_range_metadata.csv
MODIS_LST_8dayComposite_begin_end_range_metadata_final.csv


In [14]:
!ls /home/kmredilla/projects/SERDP-fish-fire/data/WRF/tsk_1km_3338/era_mean_MOD11A2

tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000049_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000057_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000065_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000073_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000081_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000089_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000097_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000105_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000113_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000121_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000129_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000137_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000145_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000153_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000161_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000169_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000177_3338.tif
tsk_8Day_daytime_wrf_era_mean_MOD11A2_2000185_3338.tif
tsk_8Day_d